<a href="https://colab.research.google.com/github/Harish-official/Grammer-Error-Detection-NLP/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [274]:
import re
import numpy as np
import pandas as pd
import nltk
#nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score
from sklearn.pipeline import Pipeline
import string

In [275]:
df = pd.read_csv('All_train_data.csv').head(100)

In [276]:
punctuations = string.punctuation
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()


In [277]:
class Predictors(TransformerMixin):
  def transform(self, Sentence, **transform_params):
    return [clean_text(text) for text in Sentence]
  def fit(self, X, y=None, **fit_params):
    return self
  def get_params(self, deep=True):
    return {}


def clean_text(text):
  return text.strip().lower()
def tokenize(sentence):
  words = nltk.word_tokenize(sentence)
  pos_tags = nltk.pos_tag(words)
  tokens = [lemmatizer.lemmatize(word).lower().strip() if(tag!='PRP' or tag!='PRP$') else word.lower().strip() for word, tag in pos_tags]
  tokens = [tok for tok in tokens if(tok not in stop_words and tok not in punctuations)]
  return tokens
  # for word, pos in pos_tags:
  #   print(word, pos)

In [278]:
vectorizer = CountVectorizer(tokenizer = tokenize, ngram_range=(1, 1))
classifier = LinearSVC()

pipe = Pipeline([('cleaner', Predictors()), ('vectorizer', vectorizer), ('classifier', classifier)])

In [279]:
X_train, X_test, y_train, y_test = train_test_split(df['input'], df['labels'], train_size=0.5, random_state=15)

In [280]:
pipe.fit(X=X_train, y=y_train)
pred_data = pipe.predict(X_test) 

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [281]:
Score = accuracy_score(y_test, pred_data)
f_score = f1_score(y_test, pred_data)
conf = confusion_matrix(y_test, pred_data)
p_score = precision_score(y_test, pred_data)
print(f"Accuracy Score is :{Score}")
print(f"F1 Score is :{f_score}")
print(f"Confusion Matrix Score is :\n{conf}")
print(f"Precision Score is :\n{p_score}")

Accuracy Score is :0.58
F1 Score is :0.43243243243243246
Confusion Matrix Score is :
[[21  4]
 [17  8]]
Precision Score is :
0.6666666666666666
